In [8]:
###우선적으로 데이터를 먼저 수집한다.
###할당을 표현할 수 있는 새로운 모델링이 필요하다.

#####필요한것 병원 설치 여부 이진변수 0,1 구하는것 병원을 x개 설치했을때 수요
####제약조건 : 거리가 30km미만이어야 함
###목적함수 : 수요 x varilable의 합
#### 병원과 지역 사이의 거리 구하기
import pandas as pd
from pulp import *

# 데이터 불러오기
df1 = pd.read_csv("C:/Users/user/Desktop/오퍼레이션/caseStudy_sup.csv", header=None)
df2 = pd.read_csv("C:/Users/user/Desktop/오퍼레이션/caseStudy_dem (2).csv", header=None)

# 입력값 설정
k = 3  # 선택 가능한 병원의 수
locations = df1.values  # 병원의 위치 좌표
demands = df2[2].tolist()  # 각 지역의 수요
coords = df2[[0, 1]].values.tolist()  # 각 지역의 위치 좌표

# 문제 정의
model = LpProblem("MaxCovering",sense= LpMaximize)

# 변수 정의 병원의 개수만큼 여부를 파악해야 하니 설정함.
variable1 = [LpVariable(f"x{i}", lowBound=0,cat="Binary") for i in range(len(locations)-1)] ###커버 가능 여부
variable2 = [LpVariable(f"y{i}", lowBound=0,cat="Binary") for i in range(len(locations)-1)] ###설치 여부

# 목적식 정의 
obj = []
for i in range(len(locations)-1):
    obj += demands[i+1] * variable1[i]  ###demand의 첫 줄이 demands라 i+1이라고 설정
model += obj

# 제약식 정의
### 수요의 최대량과 거리 설정 해야 함.
model += lpSum(variable1) == k
for j in range(len(demands)):
    model += lpSum(variable1[i] for i in range(len(locations)-1) if ((locations[i+1][0]-coords[j+1][0])**2 + (locations[i+1][1]-coords[j+1][1])**2)**0.5) <= 30000

# 문제 풀기
model.solve()

# 결과 출력
print("Selected locations:")
for i, var in enumerate(is_selected):
    if var.value() == 1:
        print(f"Hospital {i+1}: {locations[i]}")
print(f"Total demand covered: {value(prob.objective)}")

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [23]:
import pandas as pd
from pulp import *

# 데이터 불러오기
df1 = pd.read_csv("C:/Users/user/Desktop/오퍼레이션/caseStudy_sup.csv", header=None)
df2 = pd.read_csv("C:/Users/user/Desktop/오퍼레이션/caseStudy_dem (2).csv", header=None)

# 입력값 설정
k = 3  # 선택 가능한 병원의 수
locations = df1.values  # 병원의 위치 좌표
demands = df2[2].tolist()  # 각 지역의 수요
coords = df2[[0, 1]].values.tolist()  # 각 지역의 위치 좌표

# 문제 정의
model = LpProblem(name = "MaxCovering", sense =  LpMaximize)

# 변수 정의 병원의 개수만큼 여부를 파악해야 하니 설정함.
variable1 = [LpVariable(name = f"x{i+1}",lowBound= 0, cat="Binary") for i in range(len(locations)-1)]

# 목적식 정의 
obj = LpAffineExpression([(variable1[i], demands[i]) for i in range(len(locations)-1)])
model += obj
## 제약조건 정의
model += lpSum(x.values()) == k
num_constr_2 = 1  # # of constraints
constr_2 = []  # create a constraint
constr_2_coeffs = [1]
for i in range(num_constr_2):
    rhs_values = [k]
    constraint = LpConstraint(
        e=LpAffineExpression([(var_y[j], constr_2_coeffs[0]) for j in range()]),
        sense=LpConstraintLE,
        rhs=rhs_values[i],
        name=f"constraint_{num_constr_1 + i + 1}"
    )
    constr_2.append(constraint)




TypeError: '<' not supported between instances of 'str' and 'int'

In [25]:
import pandas as pd
from pulp import *
import math

# 데이터 불러오기
demand_data= pd.read_csv("C:/Users/user/Desktop/오퍼레이션/caseStudy_dem (2).csv", header=None)
supply_data = pd.read_csv("C:/Users/user/Desktop/오퍼레이션/caseStudy_sup.csv", header=None)
# 문제 정의
prob = LpProblem("Facility_Location_Problem", LpMaximize)

# 변수 정의
supply_num = len(supply_data)
demand_num = len(demand_data)

supply_list = list(range(supply_num))
demand_list = list(range(demand_num))

x = LpVariable.dicts("x", [(i,j) for i in demand_list for j in supply_list], cat='Binary')

# 목적함수 정의
prob += lpSum([x[(i,j)] for i in demand_list for j in supply_list])

# 제약식 정의
K = 99  # K값 지정
for i in demand_list:
    prob += lpSum([x[(i,j)] for j in supply_list]) <= K

for j in supply_list:
    prob += lpSum([x[(i,j)] * demand_data['demand'][i] for i in demand_list]) <= 3000

for i in demand_list:
    for j in supply_list:
        if math.sqrt((demand_data['x'][i]-supply_data['x'][j])**2 + (demand_data['y'][i]-supply_data['y'][j])**2) > 30000:
            prob += x[(i,j)] == 0

# 문제 풀이
prob.solve()
prob.writeLP("오퍼팀플과제_최종")

# 가장 많은 수요자를 커버하는 병원 출력
max_demand_covered = 0
max_supply_num = 0
for j in supply_list:
    demand_covered = 0
    for i in demand_list:
        if x[(i,j)].varValue == 1:
            demand_covered += 1
    if demand_covered > max_demand_covered:
        max_demand_covered = demand_covered
        max_supply_num = supply_data['num'][j]

# N개의 병원을 수요 커버량이 높은 순서대로 출력
N = 5 # 출력할 병원 수

supply_covered_demand = {}
for j in supply_list:
    demand_covered = 0
    for i in demand_list:
        if x[(i,j)].varValue == 1:
            demand_covered += 1
    supply_covered_demand[supply_data['num'][j]] = demand_covered

for rank, (hospital, demand_covered) in enumerate(sorted(supply_covered_demand.items(), key=lambda x: x[1], reverse=True)[:N]):
    print(f"Rank {rank+1}: Hospital {hospital} covers {demand_covered} demands.")

KeyError: 'demand'

In [ ]:
import pandas as pd
from pulp import *
import math

# 데이터 불러오기
demand_data = pd.read_csv("caseStudy_dem.csv")
supply_data = pd.read_csv("caseStudy_sup.csv")

# 문제 정의
prob = LpProblem("Facility_Location_Problem", LpMaximize)

# 변수 정의
supply_num = len(supply_data)
demand_num = len(demand_data)

supply_list = list(range(supply_num))
demand_list = list(range(demand_num))

x = LpVariable.dicts("x", [(i,j) for i in demand_list for j in supply_list], cat='Binary')

# 목적함수 정의
prob += lpSum([x[(i,j)] for i in demand_list for j in supply_list])

# 제약식 정의
K = 99  # K값 지정
for i in demand_list:
    prob += lpSum([x[(i,j)] for j in supply_list]) <= K

for j in supply_list:
    prob += lpSum([x[(i,j)] * demand_data['demand'][i] for i in demand_list]) <= 3000

for i in demand_list:
    for j in supply_list:
        if math.sqrt((demand_data['x'][i]-supply_data['x'][j])**2 + (demand_data['y'][i]-supply_data['y'][j])**2) > 30000:
            prob += x[(i,j)] == 0

# 문제 풀이
prob.solve()
prob.writeLP("오퍼팀플과제_최종")

# 가장 많은 수요자를 커버하는 병원 출력
max_demand_covered = 0
max_supply_num = 0
for j in supply_list:
    demand_covered = 0
    for i in demand_list:
        if x[(i,j)].varValue == 1:
            demand_covered += 1
    if demand_covered > max_demand_covered:
        max_demand_covered = demand_covered
        max_supply_num = supply_data['num'][j]

# N개의 병원을 수요 커버량이 높은 순서대로 출력
N = 5 # 출력할 병원 수

supply_covered_demand = {}
for j in supply_list:
    demand_covered = 0
    for i in demand_list:
        if x[(i,j)].varValue == 1:
            demand_covered += 1
    supply_covered_demand[supply_data['num'][j]] = demand_covered

for rank, (hospital, demand_covered) in enumerate(sorted(supply_covered_demand.items(), key=lambda x: x[1], reverse=True)[:N]):
    print(f"Rank {rank+1}: Hospital {hospital} covers {demand_covered} demands.")

In [5]:
import pandas as pd
from pulp import *

# 데이터 불러오기
df1 = pd.read_csv("C:/Users/user/Desktop/오퍼레이션/caseStudy_sup.csv", header=None)
df2 = pd.read_csv("C:/Users/user/Desktop/오퍼레이션/caseStudy_dem (2).csv", header=None)
# 입력값 설정
k = 3  # 선택 가능한 병원의 수
locations = df1.values.astype(float)  # 병원의 위치 좌표
demands = df2['demand'].tolist()  # 각 지역의 수요
coords = df2[['x','y']].values.astype(float).tolist()  # 각 지역의 위치 좌표

# 문제 정의
model = LpProblem("MaxCovering", sense=LpMaximize)

# 변수 정의
variable1 = [LpVariable(f"x{i}", lowBound=0,cat="Binary") for i in range(len(locations)-1)]
variable2 = [LpVariable(f"f{i}", lowBound=0,cat="Binary") for i in range(len(locations)-1)]
# 목적식 정의 
obj = 0
for i in range(len(locations)-1):
    obj += demands[i+1] * variable1[i]
model += obj

#제약식 정의
model += lpSum(variable1) == k
for i in range(len(locations)-1):
    for j in range(len(demands)-1):
        model += ((locations[i+1][0]-coords[j][0])**2 + (locations[i+1][1]-coords[j][1])**2)**0.5 * variable1[i] <= 30000

# 문제 풀기
model.solve()

# 결과 출력
print("Selected locations:")
for i, var in enumerate(variable1):
    if var.value() == 1:
        print(f"Hospital {i+1}: {locations[i+1]}")
print(f"Total demand covered: {value(model.objective)}")

ValueError: could not convert string to float: 'x'

In [ ]:
import pandas as pd
from pulp import *
import math

# caseStudy_sup.csv 파일 불러오기
hospital_data = pd.read_csv('caseStudy_sup.csv')

# caseStudy_dem.csv 파일 불러오기
demand_data = pd.read_csv('caseStudy_dem.csv')

# supply_data에서 X, Y, num 추출
x_h = hospital_data.iloc[:, 0]
y_h = hospital_data.iloc[:, 1]
num = hospital_data.iloc[:, 2]

# demand_data에서 x, y, demand 추출
x_d = demand_data.iloc[:, 0]
y_d = demand_data.iloc[:, 1]
demand = demand_data.iloc[:, 2]

# 기본 변수 저장
hospital_capacity = 3000  # 병원의 수용력
possible_distance = 30000  # 가능한 거리 ( 단위가 m라, 30000m = 30km )
hospital_distances = [[] for i in range(1,101)]
compare_list = []

# 병원과 디멘드 사이의 거리 구하기
def euclidean_distance(x1, y1, x2, y2, i, j):
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return f'{i}_{j}', distance  # 튜플 형태로 리턴

for i in range(1,len(x_d)+1):
    for j in range(1,len(x_h)+1):
        distance_name, distance = euclidean_distance(x_d[i-1], y_d[i-1], x_h[j-1], y_h[j-1], i, j)
        if distance < possible_distance:
            hospital_distances[j].append(distance_name)
            compare_list.append(distance_name)
        else:
            hospital_distances[j].append(0)

# 병원별로 커버하는 demand 계산하기
hospital_covered_demand = []
for i in range(1, len(x_h)+1):
    covered_demand = 0
    for j in range(len(hospital_distances[i])):
        if hospital_distances[i][j] != 0:
            index = int(hospital_distances[i][j].split('_')[0])-1
            covered_demand += demand[index]
    hospital_covered_demand.append((i, covered_demand))

# 커버하는 demand가 많은 병원부터 정렬하기
hospital_covered_demand.sort(key=lambda x: -x[1])

# 결과 출력
print("커버하는 demand가 많은 병원")
for i, (hospital_id, covered_demand) in enumerate(hospital_covered_demand[:99]):
    print(f"{i+1}등 병원: {hospital_id}, 커버한 demand: {covered_demand}")

In [4]:
from pulp import *
model = LpProblem(name="Maximal covering problem", sense=LpMaximize)

# Define variables
num_vars = 6
var_x = [LpVariable(name=f"x{i}", lowBound=0, cat="Binary") for i in range(1, num_vars + 1)]  # x1 to x6
var_y = [LpVariable(name=f"y{i}", lowBound=0, cat="Binary") for i in range(1, num_vars + 1)]  # y1 to y6

# Define constraints
num_constr_1 = 6  # # of constraints
constr_1 = []  # create a constraint
constr_1_coeffs = [[1, 1, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 1, 0], [0, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 1],
                   [0, 0, 0, 1, 1, 1]]
for i in range(num_constr_1):
    rhs_values = [0, 0, 0, 0, 0, 0]
    constraint = LpConstraint(
        e=LpAffineExpression([(var_y[j], constr_1_coeffs[i][j]) for j in range(num_vars)] + [(var_x[i], -1)]),
        sense=LpConstraintGE,
        rhs=rhs_values[i],
        name=f"constraint_{i + 1}"
    )
    constr_1.append(constraint)

num_constr_2 = 1  # # of constraints
constr_2 = []  # create a constraint
constr_2_coeffs = [1, 1, 1, 1, 1, 1]
for i in range(num_constr_2):
    rhs_values = [3]
    constraint = LpConstraint(
        e=LpAffineExpression([(var_y[j], constr_2_coeffs[j]) for j in range(num_vars)]),
        sense=LpConstraintLE,
        rhs=rhs_values[i],
        name=f"constraint_{num_constr_1 + i + 1}"
    )
    constr_2.append(constraint)

# Define objective function
obj_coefficients = [1500, 2000, 1700, 2010, 1300, 1000]
objective = LpAffineExpression([(var_x[j], obj_coefficients[j]) for j in range(num_vars)])
model += objective

# Add variables, constraints, and objective function to the model
for constraint in constr_1 + constr_2:
    model += constraint

# Print the LP model
print(model)
model.writeLP('out.txt')

# Solve the model
model.solve()

# Print the solution
for variable in var_x:
    print(f"{variable.name}: {variable.value()}")

for variable in var_y:
    print(f"{variable.name}: {variable.value()}")

print(f"Optimal objective value: {model.objective.value()}")

Maximal_covering_problem:
MAXIMIZE
1500*x1 + 2000*x2 + 1700*x3 + 2010*x4 + 1300*x5 + 1000*x6 + 0
SUBJECT TO
constraint_1: - x1 + y1 + y2 + 0 y3 + 0 y4 + 0 y5 + 0 y6 >= 0

constraint_2: - x2 + y1 + y2 + 0 y3 + y4 + 0 y5 + 0 y6 >= 0

constraint_3: - x3 + 0 y1 + 0 y2 + y3 + y4 + y5 + 0 y6 >= 0

constraint_4: - x4 + 0 y1 + y2 + y3 + y4 + 0 y5 + y6 >= 0

constraint_5: - x5 + 0 y1 + 0 y2 + y3 + 0 y4 + y5 + y6 >= 0

constraint_6: - x6 + 0 y1 + 0 y2 + 0 y3 + y4 + y5 + y6 >= 0

constraint_7: y1 + y2 + y3 + y4 + y5 + y6 <= 3

VARIABLES
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer
0 <= x3 <= 1 Integer
0 <= x4 <= 1 Integer
0 <= x5 <= 1 Integer
0 <= x6 <= 1 Integer
0 <= y1 <= 1 Integer
0 <= y2 <= 1 Integer
0 <= y3 <= 1 Integer
0 <= y4 <= 1 Integer
0 <= y5 <= 1 Integer
0 <= y6 <= 1 Integer

x1: 1.0
x2: 1.0
x3: 1.0
x4: 1.0
x5: 1.0
x6: 1.0
y1: 1.0
y2: 0.0
y3: 1.0
y4: 1.0
y5: 0.0
y6: 0.0
Optimal objective value: 9510.0


C:\Users\user\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
